In [1]:
import pandas as pd
import tweepy

In [2]:
#input consumer key here

consumer_key = '***************************' 
consumer_secret = '*********************************'
access_token = '***************************************'
access_token_secret = '**************************************'

In [3]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth, wait_on_rate_limit=True)   #wait on Rate-Limit!

In [4]:
#we define a function here to stream data

def get_related_tweets(key_word):
    twitter_users = []
    tweet_time = []
    tweet_string = []
    tweet_location = []
    retweet_count = []
    for tweet in tweepy.Cursor(api.search,q=key_word,count=1000,geocode='6.48939,3.37709,80km',tweet_mode='extended',include_entities=True).items(1000): #geocode here is for Lagos raduis, while count must not exceed 1000 so as not to exceed twitter limit
            if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
                if tweet.lang == "en":   #english only      
                    twitter_users.append(tweet.user.name)
                    tweet_time.append(tweet.created_at)
                    tweet_string.append(tweet.full_text)
                    tweet_location.append(tweet.user.location)
                    retweet_count.append(tweet.retweet_count)
                    #print([tweet.user.name,tweet.created_at,tweet.text])
    df = pd.DataFrame({'name':twitter_users, 'time': tweet_time, 'tweet': tweet_string, 'place':tweet_location, 'retweet-count':retweet_count})
    #df.to_csv(f"{key_word}.csv")
    return df

In [5]:
from twisted.internet import task, reactor
from datetime import datetime

timeout = 900   #15 minutes

def get_data():
    #do work here
    df1 = get_related_tweets('*')
    df2 = get_related_tweets('*')
    hold = pd.concat([df1,df2])
    hold.to_csv('./Data/df_{}.csv'.format(datetime.now().second))
    
l = task.LoopingCall(get_data)
l.start(timeout) # call every 15 minutes

reactor.run()